# Customer Data Analysis (with Python) <a class="anchor"  id="title"></a>

Joy Cheng <br>
2024-03-01 <br>
Understanding Customer and Purchase Behaviors

### 1. Problem Defining (ASK)

#### **1.1 Background**
This synthetic dataset offers information about each transaction within an online marketplace, including customer demographics such as gender and age, as well as purchase data. While it may not provide exhaustive customer details, it encompasses essential information crucial for analysis. Specifically designed to facilitate analytical and predictive purposes, it enables tasks such as trend analysis, examination of purchase behavior, market basket analysis, and customer churn prediction. By leveraging this dataset, companies and analysts can explore the dynamics of online consumer behavior, deriving valuable insights to inform strategic decision-making in e-commerce.

For this project, the main focus will be on performing Exploratory Data Analysis (EDA) from various angles, including customer demographics, products, sales timeline, etc. Additionally, based on the findings, business recommendations and marketing strategies will be provided.

#### **1.2 Potential stakeholders**
* CEO of the company
* Sales or account executives
* Marketing and product team

#### **1.3 Questions to address**
During the ask phase of data analysis, it is crucial to consider your stakeholders and determine what type of information may be most valuable to them. In real-world situations, I would suggest requesting a quick meeting with the stakeholders and relevant teams to solicit their advice and input on the matter. This approach would enable me to better comprehend the questions or challenges they've encountered and assist in refining the areas and topics I intend to explore further during the analysis.

For the purpose of this practice, I have formulated some questions that I believe would be beneficial for growing the business and gaining a better understanding of the customer pool: 

1. Are there any trends regarding customer's demographics (age/gender)?
2. Which time of the year or season tends to have the most sales? Any particular product that stands out?
3. How can we determine if a customer is high value(potential)?

### Data Preperation (PREPARE)

#### **2.1 Data source, integrity, and limitations**
The dataset, downloaded from Kaggle and offered by SHRIYASH JAGTAP, is licensed under the Open Data Commons Open Database License (ODbL). This license ensures that the dataset complies with open data principles, allowing for its unrestricted use, sharing, and modification. Furthermore, the dataset was last updated 5 months ago and has been awarded a usability score of 10.0 by Kaggle, indicating its currency and high credibility.

**About the data** 
* File name: "ecommerce_customer_data_large.csv"
* File size: 21.29 MB
* The dataset is synthetically generated using the Faker Python library and may not accurately represent real-world data.  
* It contains 13 columns with each row recording a completed transaction that includes customer information, demographics, purchase history, product information, payment mothod, return and churn statuses. Each vaiable(column) is explained in the metadata. 

**Data Limitations** 
* **Location/Country** - Information regarding the customer's location or the origin of the purchase is not provided. This would be valuable for understanding the geographical distribution of customers and identifying market opportunities in different locations.
* **Product ID/Name** - The information about what the product is for each transaction is unavailable.
* **Return details unknown** - No details regarding the quantity and price that the customer returned. 
* **Transaction ID unavailable** - Each row in the dataset represents a product transaction made by a customer. However, the absence of a transaction ID prevents us from determining whether the customer's transaction at the same time and day includes purchases of different product types. This information can only be obtained if there is a transaction ID for matching and verification. 
    * To address this concern, I began by grouping transactions based on `Customer ID` and `Purchase Date` to identify instances where a customer has purchased multiple product types on the same day. Then, I calculated the total price and quantity of products purchased on each day to compare it with the value provided in the `Total Purchase Amount` field. The result revealed that there is only one transaction per customer per day for each product type.




### Data Processing (PROCESS)

#### **3.1 Tools used for analysis**
For this project, I'll use Python, along with its Pandas and Plotly packages, to perform my analysis. Pandas streamlines the handling of 2-dimensional dataframes, structuring data akin to Excel sheets, making the process of data cleaning and manipulation more intuitive. Meanwhile, Plotly's diverse toolkit for creating customizable and interactive modern-style charts ensures a smooth and effortless data visualization experience.

#### **3.2 Data cleaning & manipulation**

In [ ]:
# intall python packages
import numpy as np # linear algebra
import pandas as pd # data processing
import plotly.express as px # create interactive charts

In [ ]:
# read file
df = pd.read_csv("/kaggle/input/e-commerce-customer-for-behavior-analysis/ecommerce_customer_data_large.csv")

df.info() # get info about count and datatypes
df.shape  # returns a tuple of total # of rows and columns
df.head() # preview first 4 rows of dataset

In [ ]:
# count distinct values in each column
df.nunique()

In [ ]:
# check duplicates, returns all duplicated rows 
df[df.duplicated(keep=False)] 

**Observations during the checking process:** 

* **Duplicates**: 
    * No duplicaded transactions were found. 
    * `Customer Age` and `Age` columns seem to contain identical data; one should be removed to avoid confusion later on.
* **Missing values**: 
    * Some missing values are present in the `Returns` column, where the count value (202,618) is less than the count of other variables (250,000). These empty cells will need to be replaced with 0 as the default, indicating that the customer has not returned the product yet. 
        * Another way for checking missing values can be using `df.isnull().sum()` to verfity and count the total number of missing values for each column.
* **Datatype**: 
    * The `Returns` data should be integers rather than floats since there are only 1s and 0s to indicate whether the purchase has been returned or not (1=returned, 0=no return). 
    * The `Purchase Date` data is currently in object type and will need to be converted into datetimes. This will help make the process of conducting time series calculations and plotting more user-friendly and efficient.
* **Errors**: 
    * The calculated total purchase amount (`Price` * `Quantity`) in each transaction appears to differ from the value provided in the `Total Purchase Amount` field. This inconsistency may indicate inaccuracies within the dataset.

After checking and identify all the issues, I then proceed to start cleaning and reformating my dataset. 

In [ ]:
# Replace all NaN to 0 for Returns
# Convert dtype to integer
df['Returns'] = df['Returns'].fillna(0).astype(int)

#check non-null count & dtype
df['Returns'].info()

In [ ]:
# Convert `Purchase Date` dtype to Datetime
df['Purchase Date'] = pd.to_datetime(df['Purchase Date'])

# Split Year and Month into new columns
df['Purchase Year'] = df['Purchase Date'].dt.year
df['Purchase Month'] = df['Purchase Date'].dt.month_name()

df[['Purchase Date','Purchase Year','Purchase Month']]

In [ ]:
# Re-calculate total purchase amount
# Remove incorrect `Total Purchased Amount` column
df['Total Price'] = df['Product Price'] * df['Quantity']
#df = df.drop(['Total Purchase Amount'], axis=1)

# Re-order columns & exclude `Total Purchased Amount` and `Age`
cols = ['Customer ID','Customer Name','Purchase Date','Purchase Year','Purchase Month','Product Category','Product Price','Quantity','Total Price','Payment Method','Returns','Gender','Customer Age','Churn']
df = df[cols]
df.info()

In [ ]:
# Save cleaned dataset to a csv file or copy to a new dataframe
df_2 = df.copy()
#df.to_csv('ecommerce_data_cleaned.csv', index=False)

After checking and addressing all missing values, duplicates, and inconsistent formatting, I now have a clean dataset ready for further exploration and analysis. Before proceeding to the next phase of the project, I will save and create a copy of this cleaned and organized dataset in a new file or dataframe. This ensures that I can easily refer back to the cleaned dataset as needed in the future.

### Data Exploration & Visualization (ANALYZE & SHARE)

### **4.1 Basic data summary & stats**


In [ ]:
# Get an overall stats of the data
df_2.describe(include='all')

In [ ]:
# Summarize object variables
df_2.describe(include=['object'])

**Overview of the dataset:**

* There are a total of 49,661 unique customers in the dataset. This count assumes that customers with the same name but different customer IDs are considered as separate individuals.
* There are more male customers than females. 
* The majority of customers are around the age of 40. 
* Among the four product types, Electronics is the most popular. 
* Most customers prefer to pay via credit cards. 

### **4.2 Explore demographics - Gender & Age**

#### **I. Trends about Gender**
1. Gender distribution 
2. Purchase price distribution by gender
3. Total & average price spent by gender and year
4. Churn and returns by genger


In [ ]:
df_2.head()

In [ ]:
# Overview stats of Gender and total price spent
gender_summary = df_2.groupby('Gender').agg(total_count=('Customer ID','size'),
                                            min_amount=('Total Price', 'min'),
                                            max_amount=('Total Price', 'max'),
                                            median_amount=('Total Price','median'),
                                            total_amount=('Total Price','sum'),
                                            average_amount =('Total Price','mean'),
                                            total_returned=('Returns','sum'),
                                            return_rates=('Returns','mean'),
                                            total_churned=('Churn','sum'),
                                            churn_rates=('Churn','mean')).reset_index()

gender_summary

In [ ]:
## 1. Gender distribution 
# calculate count for each gender
gender_count = df_2['Gender'].value_counts()
# create pie chart
fig1 = px.pie(values=gender_count, names=gender_count.index,
              color = gender_count.index,
              color_discrete_map={'Female':'#FF6692', 'Male':'#3366CC'},
              title='1. Gender Distribution')
# adjust in-graph text
fig1.update_traces(textposition='inside', textinfo='text', 
                   texttemplate='%{label}<br>%{value}<br>(%{percent})')
# adjust chart size and remove legend
fig1.update_layout(title={'x': 0.5, 'y': 0.9},
                   width=400, height=400, showlegend=False)


## 2. Purchase price distribution by gender
# Calculate the percentage of total purchased amount for each gender
sum_amount = gender_summary['total_amount'].sum()
gender_summary['total_amount_percentage'] = (gender_summary['total_amount']/sum_amount*100).round(2)

# create a bar chart 
fig2 = px.bar(gender_summary, x='total_amount',y='Gender',
              orientation='h',
              color='Gender',
              color_discrete_map={'Female': '#FF6692', 'Male': '#3366CC'},
              title='2. Total Purchase Amount by Gender',
              text=gender_summary.apply(lambda x: f"${x['total_amount']:,}<br>({x['total_amount_percentage']:.2f}%)", axis=1),
              labels={'total_amount':'Total Purchased Amount ($)'})
fig2.update_layout(title={'x': 0.5, 'y': 0.9},
                  yaxis_title= None, 
                  width=550, height=350, showlegend=False)


## 3 Average price spent breakdown by year
# Calculate the overall average spending and prepare data for plotting
overall_avg_price = df_2['Total Price'].mean()
avg_price_gender_year = df_2.groupby(['Gender', 'Purchase Year'], as_index=False)['Total Price'].mean()

# Create box plot visualizing total price by gender and year, including overall average line
fig3 = px.box(df_2, x='Gender', y='Total Price', color='Purchase Year',
              title='3. Total Price Spent by Gender, Breakdown by Year',
              labels={'Total Price': 'Total Purchased Amount ($)'})

fig3.add_hline(y=overall_avg_price, line_dash="dash", line_color="#325A9B",
               annotation_text=f'Overall Average: ${overall_avg_price:.2f}',
               annotation_position="top",  # Adjusted for better visibility
               annotation_font_color="#325A9B")

fig3.update_layout(title={'x': 0.5, 'y': 0.9}, 
                   width=700, height=450,
                   xaxis_title=None)

## 4. Customer churn and returns by gender
# Convert churn and return rates to percentage
gender_summary['churn_rates'] *= 100
gender_summary['return_rates'] *= 100


fig4 = px.bar(gender_summary, x='Gender', y=['total_churned', 'total_returned'],
             barmode='group', 
             title='4. Customer Churn and Returns by Gender',
             text_auto='.4s',
             labels={'value': 'Number of Customers'},
             hover_data={'churn_rates': True, 'return_rates': True}) 

# Adjust data shown in the hover text
fig4.update_traces(hovertemplate="Gender: %{x}<br>"
                                 "Total: %{y:,.0f}<br>"
                                 "Churn Rate: %{customdata[0]}%<br>"
                                 "Return Rate: %{customdata[1]}%<extra></extra>")

# Add annotations for churn/return rates
for index, row in gender_summary.iterrows():
    fig4.add_annotation(x=row['Gender'], y=row['total_churned'],
                        text=f"{row['churn_rates']:.2f}%", 
                        showarrow=False, xanchor='right', yanchor='bottom', 
                        font=dict(color='blue', size=12))

for index, row in gender_summary.iterrows():
    fig4.add_annotation(x=row['Gender'], y=row['total_returned'],
                        text=f"{row['return_rates']:.2f}%",
                        showarrow=False, xanchor='left', yanchor='bottom', 
                        font=dict(color='red', size=12))
    

fig4.update_layout(title={'x': 0.5, 'y': 0.9},
                   width=600, height=500, 
                   xaxis_title= None,
                   legend_title=None,
                   legend=dict(orientation="h",yanchor="bottom",
                              y=1.02, xanchor="center",x=0.5))

# display graphs
fig1.show()
fig2.show()
fig3.show()
fig4.show()

> **Gender distribution and trends:**
> * Both male and female customers exhibit similar average spending, approximately $$756, suggesting a diverse customer base.
> * The average annual expenditure from 2020 to 2023 remains stable for both genders, hovering around $2,725, with males slightly outspending females by 0.11%.
> * The presence of outliers beyond the upper fence and the gap between average and median points in the box plots indicates the existence of outliers in the dataset.These outliers may require further investigation to understand if they represent rare scenarios, errors in data entry or calculation, or other factors, influencing the decision to exclude or explore them for additional insights.
> * Both churn rates and return rates are similar for both genders, with a slight variation—females shows a slighly higher churn rate, and males show a marginally higher return rate. However, both of these differences are minimal and do not necessarily indicate that one is more likley to churn or return than the other. Nevertheless, both churn and return rates are unusually high and evenly distributed across genders, which is atypical in consumer behavior patterns. It is advisable to investigate the causes of these trends further to determine if there are any inaccuracies or errors in the data.

#### **II. Trends about Age**
1. Age distribution
2. Compare age distribution by gender
3. Average price spent by age range
4. Churned rate by age group

First, start with defining the age range into <br>
`Age Range: 0-9, 10-19, 20-29, 30-39, 40-49, 50-59, 60-69, and 70+`

In [ ]:
# Define bins for age groups
age_bins = [0,9,19,29,39,49,59,69,np.inf]
age_labels = ['0-9','10-19','20-29','30-39','40-49','50-59','60-69','70+']

# Add a new column and assign age group category to each transaction in df_2
df_2['Age Group'] = pd.cut(df_2['Customer Age'], bins=age_bins, labels=age_labels) 

# Check if `Age Group` is assigned correctly and added to the dataset
df_2[['Customer ID','Customer Age','Age Group']].sort_values('Customer Age', ascending=False)

#df_2[df_2['Age Group'].isnull()] # check any missing values

In [ ]:
## 1. Age distribution by age
# Ensure the 'Age Group' in df_2 is categorized according to age_labels
# this will help to connect the data points in the correct order in line chart
df_2['Age Group'] = pd.Categorical(df_2['Age Group'], categories=age_labels, ordered=True)

# Count each age group
age_group_count = df_2['Age Group'].value_counts().sort_index().reset_index(name='Count')

# Calculate the percentage for each age group
age_group_count['Percentage'] = round(age_group_count['Count'] / age_group_count['Count'].sum()*100, 2)
age_group_count['Percentage Text'] = age_group_count['Percentage'].astype(str) + '%'

# Plot a line chart with bars
fig1 = px.line(age_group_count,x='Age Group', y='Count',
               title='1. Age Distribution',
               markers=True,
               text='Percentage Text',
               labels={'Count': 'Number of Customers'})

fig1.update_traces(textposition="top center") #adjust text position for percentages 
fig1.add_bar(x=age_group_count['Age Group'], y=age_group_count['Count'], text=age_group_count['Count'], textposition='inside', name='Count')
fig1.update_traces(textfont_color='white', selector=dict(type='bar'))
fig1.update_layout(title={'x': 0.5, 'y': 0.9},
                   xaxis_title= None,
                   yaxis_title= 'Count',
                   width=750, height=450, 
                   showlegend=False)


## 2. Age distrbution by gender
# count each age group by gender
age_group_gender_count = df_2[['Gender','Age Group']].value_counts().reset_index(name='Count')

fig2 = px.bar(age_group_gender_count, x='Age Group', y='Count', 
              color='Gender', barmode='group', 
              color_discrete_map={'Female': '#FF6692', 'Male': '#3366CC'},
              category_orders={'Age Group': age_labels},
              title='2. Age Distribution by Gender',
              text_auto=True)
fig2.update_layout(title={'x': 0.5, 'y': 0.9},
                   width=750, height=450,
                   xaxis_title= None,
                   legend_title=None,
                   legend=dict(orientation="h",yanchor="bottom",
                               y=1.02, xanchor="center",x=0.5))


## 3. Average price spent by age group
# calculate overall and average of total spending for each age group
overall_avg_price = df_2['Total Price'].mean()
avg_price_age_group = df_2.groupby(['Age Group'])['Total Price'].mean().reset_index(name='Avg Price')

fig3 = px.bar(avg_price_age_group, x= 'Age Group', y= 'Avg Price', 
              color = 'Avg Price',
              color_continuous_scale= "peach", 
              category_orders={'Age Group': age_labels},
              title='3. Average Price Spent by Age Group',
              text= [f"${value:.2f}" for value in avg_price_age_group['Avg Price']],
              hover_data={'Age Group':True, 'Avg Price':"${:.2f}"})

# Adjust data and format shown in the hover data
fig3.update_traces(hovertemplate="Age Group: %{x}<br>"
                                 "Average Spending: %{y:$,.2f}")

# Add a line to show the overall average for spending 
fig3.add_hline(y=overall_avg_price, line_dash="dash", line_color="#1CBE4F",
               annotation_text=f'Overall Average: ${overall_avg_price:.2f}',
               annotation_font_color="#1CBE4F",
               annotation_position="top left")

fig3.update_layout(title={'x': 0.5, 'y': 0.9},
                   width=700, height=450,
                   xaxis_title= None, 
                   yaxis_title= 'Average Spending ($)',
                   yaxis=dict(range=[500, 800],  # Set range from 400 to 800
                              tick0=500,  # Set starting tick at 400
                              dtick=50),  # Set interval between ticks
                   showlegend=False)


## 4. Customer churn and returns by age group
# Group by age group and churn/return status
age_group_churned_returned = df_2.groupby('Age Group').agg(total_churned=('Churn','sum'),
                                                           churn_rates=('Churn','mean'),
                                                           total_returned=('Returns','sum'),
                                                           return_rates=('Returns','mean')).fillna(0).reset_index()

# Calculate the total churned/returned count and churn/return rates 
age_group_churned_returned['churn_rates'] *= 100
age_group_churned_returned['return_rates'] *= 100


fig4 = px.bar(age_group_churned_returned, x='Age Group', y=['total_churned', 'total_returned'],
             barmode='group', 
             category_orders={'Age Group': age_labels},
             title='4. Customer Churn and Returns by Age Group',
             text_auto= '.3s' ,
             labels={'value': 'Number of Customers'},
             hover_data={'churn_rates': True, 'return_rates': True}) 

# Adjust data shown in the hover text
fig4.update_traces(hovertemplate="Age Group: %{x}<br>"
                                 "Total: %{y:,.0f}<br>"
                                 "Churn Rate: %{customdata[0]}%<br>"
                                 "Return Rate: %{customdata[1]}%<extra></extra>")

fig4.update_traces(textfont={'size': 11}, textposition='inside')

# Add annotations for churn rates
for index, row in age_group_churned_returned.iterrows():
    fig4.add_annotation(x=row['Age Group'], y=row['total_churned'],
                        text=f"{row['churn_rates']:.1f}%", 
                        showarrow=False, xanchor='right', yanchor='bottom', 
                        font=dict(color='blue', size=11))

# Add annotations for return rates
for index, row in age_group_churned_returned.iterrows():
    fig4.add_annotation(x=row['Age Group'], y=row['total_returned'],
                        text=f"{row['return_rates']:.1f}%",
                        showarrow=False, xanchor='left', yanchor='bottom', 
                        font=dict(color='red', size=11))

fig4.update_layout(title={'x': 0.5, 'y': 0.9},
                   width=800, height=500, bargap=0.1, 
                   xaxis_title= None,
                   legend_title=None,
                   legend=dict(orientation="h",yanchor="bottom",
                              y=1.02, xanchor="center",x=0.5))


# display graphs
fig1.show()
fig2.show()
fig3.show()
fig4.show()

> **Distribution and trends by Age:**
> * The largest age group appears to be between 20-29 years old, closely followed by the 30-39 age group. However, there is a noticeable drop in the number of customers in the age groups of 70+ and 10-19.
> * Across genders, there seems to be a relatively similar distribution of age groups, with slight variations in the counts. Males outnumber females in almost all age groups except for the 20-29 age group.
> * An interesting observation is that the average spending tends to increase as age increases.
> * The age groups 20-29 and 60-69 appear to have higher churn and return rates, suggesting that customers within these age brackets are more likely to both return items and discontinue their membership. This trend could be influenced by specific needs or preferences prevalent in these age demographics. 
> * The graph also suggests a potential correlation between return rates and churn rates: as return rates increase, churn rates also tend to rise. However, further investigation is necessary to validate this observation conclusively.



### **4.3 Explore Trends About Products & Sales**
1. Product distribution by category 
2. Product category total count vs. returns
3. Product sales revenue by year
4. Product sales revenue by gender
5. Product sales revenue by age group

In [ ]:
# To include stats abour product reutrns, 
# we'll need to calculate the Return_Quantity when Returns == 1  
# (assuming all items are returned altogether for each transaction)
df_2['Return Quantity'] = df_2.apply(lambda row: row['Quantity'] if row['Returns'] == 1 else 0, axis=1)

product_summary_table = df_2.groupby('Product Category').agg(Count=('Product Category', 'size'),
                                               Order_Quantity=('Quantity', 'sum'),
                                               Total_Revenue=('Total Price','sum'),
                                               Average_Revenue=('Total Price','mean'),
                                               Return_Count=('Return Quantity', 'sum')).reset_index()
product_summary_table

In [ ]:
# 1. Product Distribution by category
# Calculate the overall percentage for product quantity sold
total_order_quantity = product_summary_table['Order_Quantity'].sum()
product_summary_table['Overall_Percentage'] = (product_summary_table['Order_Quantity'] / total_order_quantity) * 100

# Calculate the percentage for return
product_summary_table['Return_Rate'] = round((product_summary_table['Return_Count'] / product_summary_table['Order_Quantity']) * 100, 2)

product_summary_table

In [ ]:
# 1. Product Distribution and Returns by category
# Set the category order for plotting colors
category_order = ["Electronics", "Clothing", "Home", "Books"]

fig1 = px.bar(product_summary_table, 
             x='Product Category', 
             y=['Order_Quantity', 'Return_Count'],
             barmode='overlay', opacity=0.7,
             color='Product Category',
             color_discrete_sequence=px.colors.qualitative.T10,
             pattern_shape = 'variable',
             category_orders={'Product Category': category_order},
             title='Product Category Distribution vs. Returns',
             labels={'value': 'Return & Total Order Count'})

# Add overall percentage and returns rate as text on the graph
for i, row in product_summary_table.iterrows():
    fig1.add_annotation(x=row['Product Category'], y=row['Order_Quantity']-15000, text=f"Total:{row['Order_Quantity']/ 1000:,.2f}K<br>({row['Overall_Percentage']:.2f}%)",
                       font=dict(color='black', size=12), showarrow=False)

for i, row in product_summary_table.iterrows():
    fig1.add_annotation(x=row['Product Category'], y=row['Return_Count']-15000, text=f"Returned:{row['Return_Count']/ 1000:,.2f}K<br>({row['Return_Rate']:.2f}%)",
                       font=dict(color='white', size=12), showarrow=False)
    

fig1.update_layout(title={'x': 0.5, 'y': 0.9},
                  xaxis_title= None,
                  width=700, height=500,
                  showlegend=False)



# 2. Sales Revenue by Product Category
total_revenue = df_2.groupby(['Product Category','Purchase Year'])['Total Price'].sum().reset_index()
total_revenue['Purchase Year'] = total_revenue['Purchase Year'].astype(str)

fig2 = px.bar(total_revenue, x='Product Category', y='Total Price',
              color='Purchase Year', barmode= 'group',
              color_discrete_sequence=px.colors.qualitative.Set2,
              category_orders={'Product Category': category_order},
              title = '2. Sales Revenue by Product Category and Year',
              labels={'Total Price': 'Total Price ($)'})
fig2.update_traces(texttemplate='%{y:$.4s}', textposition='inside', textangle=90)
fig2.update_layout(title={'x': 0.5, 'y': 0.9},
                  xaxis_title= None,
                  width=700, height=500)


# 3. Product revenue by Gender
product_revenue_gender = df_2.groupby(['Product Category','Gender'])['Total Price'].sum().reset_index(name='Total Revenue')

fig3 = px.bar(product_revenue_gender,x='Product Category', y='Total Revenue', 
              color='Gender',
              color_discrete_map={'Female': '#FF6692', 'Male': '#3366CC'},
              barmode='group', 
              title='3. Sales Revenue by Product Category & Gender',
              category_orders={'Product Category': category_order},
              labels={'Total Revenue': 'Total Revenue ($)'})

fig3.update_traces(texttemplate='%{y:$.4s}', textposition='inside', textangle=90)

fig3.update_layout(title={'x': 0.5, 'y': 0.9},
                  xaxis_title= None,
                  width=700, height=500,
                  legend_title=None,
                  legend=dict(orientation="h",yanchor="bottom",
                              y=1.02, xanchor="center",x=0.5))


# 4. Product revenue by Age Group
product_revenue_age = df_2.groupby(['Product Category','Age Group'])['Total Price'].sum().reset_index(name='Total Revenue')

fig4 = px.bar(product_revenue_age[product_revenue_age['Total Revenue'] > 0], #filter out empty bars
              x='Age Group', y='Total Revenue',
              color='Product Category', barmode= 'group',
              color_discrete_sequence=px.colors.qualitative.T10,
              category_orders={'Product Category': category_order},
              title = '4. Product Sales Revenue by Category & Age Group',
              labels={'Total Revenue': 'Total Revenue ($)'})

fig4.update_traces(texttemplate='%{y:$.3s}', textangle=90)

fig4.update_layout(title={'x': 0.5, 'y': 0.9},
                  xaxis_title= None,
                  width=800, height=500,
                  legend_title=None,
                  legend=dict(orientation="h",yanchor="bottom",
                              y=1.02, xanchor="center",x=0.5))


fig1.show()
fig2.show()
fig3.show()
fig4.show()

> **Product distribution and sales trends:**
> * Clothing has the highest demand of all categories at 25.12% while Books has the highest return rate at 41%. 
> * The overall return rates for all product types appear abnormally high and even, which would merit further investigation into the underlying causes.
> * Home and Clothing consistently generate the highest sales revenue year over year, while Books register the lowest sales revenue among the categories.
> * It appears that customers aged 20-29 show a higher preference for Home goods, while those in the 40-49 age group tend to favor Clothing more. Other age groups show a more even distribution among all product types.

### **4.4 Explore Time Series Pattern**
1. Monthly count of orders by year
2. Compare current and previous annual sales revenue
3. Monthly return and churn rates over time

In [ ]:
# Quick overview of yearly sales revenue
annual_sales_summary = df_2.groupby('Purchase Year')['Total Price'].agg(['min', 'max','mean','median','sum',])
annual_sales_summary

In [ ]:
# Set month names in order and dtype to category
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 
               'July', 'August', 'September', 'October', 'November', 'December']

df_2['Purchase Month'] = pd.Categorical(df_2['Purchase Month'], 
                                        categories=month_order, 
                                        ordered=True)

# Calculate the total and average of monthly sales revenue
monthly_sales = df_2.groupby(['Purchase Year', 'Purchase Month']).agg(Count=('Customer ID', 'size'),
                                                                      Total_Revenue=('Total Price', 'sum'),
                                                                      Avg_Revenue=('Total Price', 'mean')).reset_index()

In [ ]:
## 1. Monthly count of orders, split by years
# Convert years to string for bar chart
monthly_sales['Purchase Year'] = monthly_sales['Purchase Year'].astype(str)

fig1 = px.bar(monthly_sales, x='Purchase Month', y='Count', 
               color='Purchase Year', facet_row='Purchase Year',
               title='1. Number of Orders per Month (by Year)', 
               labels={'Count': 'Order Count', 'Purchase Year': 'Year'},
               hover_data={'Count': ':,.0f'},
               text_auto=True)

fig1.update_layout(title={'x': 0.5, 'y': 0.9},
                  xaxis_title= None, 
                  width=650, height=600,
                  showlegend=False)

## 2. Compare total monthly sales
# Filter out data points where 'Total_Revenue' is 0
fig2 = px.area(monthly_sales[monthly_sales['Total_Revenue'] != 0], x='Purchase Month', y='Total_Revenue', 
              color='Purchase Year', facet_col='Purchase Year',
              color_discrete_sequence=px.colors.qualitative.Dark2,
              title='2. Total Monthly Sales Revenue (by Year)',
              labels={'Total_Revenue': 'Total Revenue ($)', 'Purchase Year': 'Year'},
              hover_data={'Total_Revenue': ':$,.0f'},
              markers=True)

fig2.update_yaxes(tickprefix="$")

fig2.update_layout(title={'x': 0.5, 'y': 0.9},
                  xaxis_title= None,
                  xaxis2_title= None,
                  xaxis3_title= None,
                  xaxis4_title= None,
                  width=800, height=350,
                  showlegend=False)

## 3. Compare average monthly sales
fig3 = px.line(monthly_sales, x='Purchase Month', y='Avg_Revenue', 
              color='Purchase Year', facet_col='Purchase Year',
              color_discrete_sequence=px.colors.qualitative.Set2,
              title='3. Average Monthly Sales Revenue (by Year)',
              labels={'Avg_Revenue': 'Average Revenue ($)', 'Purchase Year': 'Year'},
              hover_data={'Avg_Revenue': ':$,.0f'},
              markers=True)

fig3.update_yaxes(tickprefix="$")

fig3.update_layout(title={'x': 0.5, 'y': 0.9},
                  xaxis_title= None,
                  xaxis2_title= None,
                  xaxis3_title= None,
                  xaxis4_title= None,
                  width=800, height=350,
                  yaxis=dict(range=[700, 840]), 
                  showlegend=False)


## 4. Monthly return and churn rate by year    
df_return_churn = df_2[['Customer ID','Purchase Date','Returns','Churn']]
df_return_churn['Purchase Date'] = df_return_churn['Purchase Date'].dt.strftime('%Y-%m')
rates = df_return_churn.groupby('Purchase Date').agg(
    Return_Rate=('Returns', 'mean'), Churn_Rate=('Churn', 'mean')).reset_index()

# Convert rates to percentage
rates['Return_Rate'] *= 100
rates['Churn_Rate'] *= 100

fig4 = px.line(rates, x='Purchase Date', y=['Return_Rate','Churn_Rate'], 
              title='4. Monthly Return and Churn Rates Over time',
              markers=True,
              hover_data={'variable':':name'})


fig4.update_traces(hovertemplate="Type: %{customdata}<br>"
                                 "Purchase Date: %{x|%Y-%m}<br>"
                                 "Rate: %{y:.2f}%")

# Calculate the positions for annotations
max_return_rate = rates.loc[rates['Return_Rate'].idxmax()]
min_return_rate = rates.loc[rates['Return_Rate'].idxmin()]
max_churn_rate = rates.loc[rates['Churn_Rate'].idxmax()]
min_churn_rate = rates.loc[rates['Churn_Rate'].idxmin()]

annotations = [
    {'x': max_return_rate['Purchase Date'], 'y': max_return_rate['Return_Rate']+1, 
     'text': f"highest: {max_return_rate['Return_Rate']:.2f}%", 'showarrow': False},
    {'x': min_return_rate['Purchase Date'], 'y': min_return_rate['Return_Rate']-1, 
     'text': f"lowest: {min_return_rate['Return_Rate']:.2f}%", 'showarrow': False},
    {'x': max_churn_rate['Purchase Date'], 'y': max_churn_rate['Churn_Rate']+1, 
     'text': f"highest: {max_churn_rate['Churn_Rate']:.2f}%", 'showarrow': False},
    {'x': min_churn_rate['Purchase Date'], 'y': min_churn_rate['Churn_Rate']-1, 
     'text': f"lowest: {min_churn_rate['Churn_Rate']:.2f}%", 'showarrow': False} ]

for annotation in annotations:
    fig4.add_annotation(x=annotation['x'], y=annotation['y'], text=annotation['text'], 
                        showarrow=annotation['showarrow'], font=dict(size=11))

fig4.update_layout(title={'x': 0.5, 'y': 0.9},
                  xaxis_title= None,
                  yaxis_title= 'Rates (%)',
                  width=750, height=450,
                  legend_title=None,
                  legend=dict(orientation="h",yanchor="bottom",
                              y=1.02, xanchor="center",x=0.5))   


#display all charts
fig1.show()
fig2.show()
fig3.show()
fig4.show()

> **Patterns about sales and return/churn over time:**
> * The final transaction date recorded in the dataset is September 13, 2023, which accounts for the notably lower total annual revenue compared to previous years.
> * The annual sales revenue across all years appears consistent with minor fluctuations, typically with February having the lowest total sales, likely due to having fewer days in the month. September seems to be a slower month in the year, experiencing a drop in revenue after August.
> * Although February consistently shows the lowest sales figures for all years, it does not have the lowest average monthly sales except for 2023.
> * The return rate remains relatively stable over time, hovering around 40%. The number seems high and should require further investigation into the type of products being returned as well as customer's satisfaction to help reduce the rate. 
> * The churn rate shows slight fluctuations but remains under 22% through the years. 
 

### **4.5 Customer segmentation & ranking**
Segment customers by customer's individual Customer Lifetime Value(CLV) based on purchase history into High, Medium, and low tiers. Using the formula below: 

**Customer Lifetime Value (CLV)** = Customer Value x Average Customer Lifespan <br>
> **Average Purchase Value** = Total Revenue / Number of Orders <br>
> **Average Purchase Frequency** = Total Number of Purchases per Customer / Number of Customers <br>
> **Average Customer Lifespan** = Sum of Customer Lifespans / Number of Customers <br>
> **Customer Value** = Average Purchase Value x Average Purchase Frequency <br>

Next, utilize the calculated CLV to help identify the most valuable customers and further explore and understand their purchasing behaviors by comparing:

1. Customer segment distribution
2. Total sales revenue
3. Monthly sales revenue (by year)
4. Age distributions
5. Churn and return rates

In [ ]:
# Calculate each customer's CLV
# Start with grouping data by customer's ID and sum their total order num and purchase amount
customer_df = df_2.groupby(['Customer ID','Customer Name']).agg(
                order_count=('Customer ID', 'size'), 
                total_revenue=('Total Price', 'sum'),
                avg_purchase_value=('Total Price','mean'),
                return_num=('Returns', 'sum'),
                churned=('Churn','sum')).reset_index()

# Re-assign churned values into boolean 1,0 based on the condition
customer_df['churned'] = customer_df['churned'].apply(lambda x: 1 if x >= 1 else 0)
customer_df

In [ ]:
# Calculate average purchase frequency (APF) per customer
customer_df['avg_purchase_freq'] = customer_df['order_count'] / len(customer_df)

# Calculate the average customer lifespan
# Calculate the difference between each customer's the first and last purchase date
customer_lifespans = df_2.groupby('Customer ID')['Purchase Date'].apply(lambda x: (x.max() - x.min()).days)
avg_lifespan_days = round(customer_lifespans.mean(), 1)

print(f"Customer Average Lifespan: {avg_lifespan_days:.1f} days")

In [ ]:
# Calculate customer value for each customer
customer_df['customer_value'] = customer_df['avg_purchase_value'] * customer_df['avg_purchase_freq']

# Then calculate the final Customer Lifetime Value (CLV)
customer_df['CLV'] = customer_df['customer_value'] * avg_lifespan_days

customer_df.head()

In [ ]:
# Re-order columns and create a summary stat for CLV
customer_df = customer_df[['Customer ID','Customer Name','churned','return_num','order_count',
                           'total_revenue','avg_purchase_value','avg_purchase_freq',
                           'customer_value','CLV']]
customer_df.describe()

In [ ]:
# Segment customers into High, Medium, and Low
# Define cutoff points by 25th and 75th percentile
clv_summary = customer_df['CLV'].describe()
high_cutoff = clv_summary['75%']  # Extract data point for 75th percentile
low_cutoff = clv_summary['25%']   # Extract data point for 25th percentile

# Assign segment labels based on CLV values
def assign_segment(clv):
    if clv >= high_cutoff:
        return 'High'
    elif clv >= low_cutoff:
        return 'Medium'
    else:
        return 'Low'

# Apply segment assignment to each row in the DataFrame
customer_df['Segment'] = customer_df['CLV'].apply(assign_segment)

customer_df.sort_values(by='CLV', ascending=False).head(10)

Now that we have all the customer's CLV calculated and segments assigned, we can use them to find out who our top 10 high-value customers are. We'll use Pandas embeded SQL to perform the query:

In [ ]:
# Use query to find out the top 10 high-value customers 
# exclude ones who has churned
from pandasql import sqldf

print('* Top 10 High-Value Customers (Current) *')
print('')
print(sqldf('''SELECT DISTINCT `Customer ID`, `Customer Name`, CLV
                 FROM customer_df 
                WHERE Segment = 'High' AND churned == 0
                ORDER BY CLV DESC
                LIMIT 10'''))

In [ ]:
# Also apply segment-tiers to df_2 for later use in plotting
# use merge to join data based on `Customer ID` in customer_df
df_2 = df_2.merge(customer_df[['Customer ID','Segment']], on='Customer ID', how='left')

# Rename the 'Segment' column to avoid conflicts
df_2.rename(columns={'Segment':'Segment Tier'}, inplace=True)

# Check for any missing values after join
df_2.info()

In [ ]:
# Create an overall summary table for segment
# Calculate the total count/order/sales/churn and averages 
segment_summary = customer_df.groupby('Segment').agg(Total_Count=('Segment', 'size'),
                                               Total_Order_Count=('order_count', 'sum'),
                                               Total_Revenue=('total_revenue','sum'),
                                               Average_Revenue=('total_revenue','mean'),
                                               Total_Churned=('churned', 'sum'),
                                               Churn_Rates=('churned','mean'),
                                               Total_Return_Num=('return_num','sum')).reset_index()
# Calculate return rates for each segment
segment_summary['Return_Rates'] = segment_summary['Total_Return_Num'] / segment_summary['Total_Order_Count']

segment_summary

In [ ]:
## 1. Customer segment distribution
# Set the segment order for plotting colors
segment_order = ["Low", "Medium", "High"]

fig1 = px.pie(segment_summary, values='Total_Count', names='Segment',
              color = 'Segment',
              color_discrete_sequence=px.colors.qualitative.Dark24,
              category_orders={'Segment': segment_order},
              title='1. Customer Segment Distribution')
fig1.update_traces(textposition='inside', textinfo='text', 
                   texttemplate='%{label}<br>%{value}<br>(%{percent})')
fig1.update_layout(title={'x': 0.5, 'y': 0.9},
                   width=400, height=400, showlegend=False)


## 2. Total purchase amount over time by segment
# Sum all revenues to calculate the spending percentage for each segment
sum_overall_revenue = segment_summary['Total_Revenue'].sum()
segment_summary['Revenue Percentage'] = (segment_summary['Total_Revenue'] / sum_overall_revenue) * 100

fig2 = px.bar(segment_summary, x='Segment', y='Total_Revenue', 
              color='Segment', 
              color_discrete_sequence=px.colors.qualitative.Dark24,
              title='2. Total Purchase Amount by Segment',
              labels={'Total_Revenue': 'Total Money Spent ($)'},
              category_orders={'Segment': segment_order},
              text=segment_summary.apply(lambda x: f"${x['Total_Revenue']:,}<br>({x['Revenue Percentage']:.2f}%)", axis=1),
              hover_data={'Total_Revenue': ':$,.0f'})
fig2.update_layout(title={'x': 0.5, 'y': 0.9},
                   width=550, height=450, showlegend=False)


## 3. Monthly Sales Revenue Over Time
# Convert `Purchase Date` to Year-Month format
# Calculate the total price by date and segment
segment_df = df_2[['Customer ID','Purchase Date','Total Price','Segment Tier']]
segment_df['Purchase Date'] = segment_df['Purchase Date'].dt.strftime('%Y-%m')
segment_monthly_sales = segment_df.groupby(['Purchase Date','Segment Tier'])['Total Price'].sum().reset_index(name='Sum')

fig3 = px.area(segment_monthly_sales, x='Purchase Date', y='Sum', 
              color='Segment Tier', 
              color_discrete_map={"Low":'#2E91E5', "Medium":'#E15F99', "High":'#1CA71C'},
              title='3. Monthly Sales Revenue Over Time by Segment',
              markers=True,
              symbol='Segment Tier',
              hover_data={'Segment Tier':':name','Purchase Date':True,'Sum':True})

fig3.update_traces(hovertemplate="Value: %{customdata}<br>"
                                "Purchase Date: %{x|%Y-%m}<br>"
                                "Sum: %{y:$,.3s}")

fig3.update_layout(title={'x': 0.5, 'y': 0.9},
                  xaxis_title= None,
                  yaxis_title= 'Total Revenue ($)',
                  width=750, height=500,
                  legend_title=None,
                  legend=dict(orientation="h",yanchor="bottom",
                              y=1.02, xanchor="center",x=0.5)) 


# 4. Age Distrbution by Segmtent
# Calculate the count for each age and segment group
segment_age_count = df_2.groupby(['Age Group','Segment Tier']).size().reset_index(name='Count')

fig4 = px.bar(segment_age_count, x='Age Group', y='Count', 
             color='Segment Tier', barmode='group',
             color_discrete_sequence=px.colors.qualitative.Dark24,
             category_orders={'Segment Tier': segment_order,'Age Group': age_labels},
             text_auto=True,
             title='4. Age Distribution by Segment')
fig4.update_layout(title={'x': 0.5, 'y': 0.9},
                   width=800, height=500)


## 5. Customer churn and returns by Segment
# Convert churn/return rates to percentage 
segment_summary['Churn_Rates'] *= 100
segment_summary['Return_Rates'] *= 100

fig5 = px.bar(segment_summary, x='Segment', y=['Total_Churned', 'Total_Return_Num'],
             barmode='group', 
             category_orders={'Segment': segment_order},
             title='5. Customer Churn and Returns by Segment',
             text_auto= '.3s' ,
             labels={'value': 'Number of Customers'},
             hover_data={'Churn_Rates': True, 'Return_Rates': True}) 

# Adjust data shown in the hover text
fig5.update_traces(hovertemplate="Segment Tier: %{x}<br>"
                                 "Total: %{y:,.0f}<br>"
                                 "Churn Rate: %{customdata[0]}%<br>"
                                 "Return Rate: %{customdata[1]}%<extra></extra>")

fig5.update_traces(textfont={'size': 11}, textposition='inside')

# Add annotations for churn rates
for index, row in segment_summary.iterrows():
    fig5.add_annotation(x=row['Segment'], y=row['Total_Churned'],
                        text=f"{row['Churn_Rates']:.2f}%", 
                        showarrow=False, xanchor='right', yanchor='bottom', 
                        font=dict(color='blue', size=11))

# Add annotations for return rates
for index, row in segment_summary.iterrows():
    fig5.add_annotation(x=row['Segment'], y=row['Total_Return_Num'],
                        text=f"{row['Return_Rates']:.2f}%",
                        showarrow=False, xanchor='left', yanchor='bottom', 
                        font=dict(color='red', size=11))

fig5.update_layout(title={'x': 0.5, 'y': 0.9},
                   width=600, height=500, 
                   xaxis_title= None,
                   legend_title=None,
                   legend=dict(orientation="h",yanchor="bottom",
                              y=1.02, xanchor="center",x=0.5))


###
fig1.show()
fig2.show()
fig3.show()
fig4.show()
fig5.show()

> **Insights discovered from customer segmentation:**
> * The medium segment holds the majority of customers, comprising 50% of the customer base, and contributes significantly to the total purchase amount, accounting for 47% of the total revenue, which amounts to 89 million dollars. It is closely followed by the high segment, contributing 44% of the total revenue, equivalent to 84 million dollars.
> * There appears to be stability and consistency in revenue generated by each segment over time. This suggests a reliable customer base within each segment. All three segments exhibit similar revenue patterns and fluctuations, yet the medium segment generally generates higher revenue whereas the low segment generates the least revenue at all times. 
> * The age distribution across segments is notably similar, with the majority of customers falling within the 20-69 age range. Specifically, the age group of 20-29 shows the highest concentration across all segments.
> * Churn rates and return rates across all three segments hover around 20% and 40%, respectively, with the medium segment exhibiting the highest churn and return rates. This indicates similarity in customer retention and return challenges across segments.

### **4.6 Payment Analysis**
1. Payment method distribution vs. returns
2. Payment method distribution by gender and age group
3. Payment method distribution by segment


In [ ]:
payment_count_returns = df_2.groupby('Payment Method').agg(total_count=('Customer ID','size'),
                                                     total_return=('Returns','sum'),
                                                     return_rates=('Returns','mean')).reset_index()

payment_count_returns['percentage'] = payment_count_returns['total_count']/payment_count_returns['total_count'].sum()*100
payment_count_returns['return_rates'] *= 100

payment_count_returns[['Payment Method','total_count','percentage','total_return','return_rates']]

In [ ]:
## 1. Payment method distribution vs. returns
payment_method_order = ['Cash', 'Credit Card','PayPal']

fig1 = px.bar(payment_count_returns, x='Payment Method', y=['total_count', 'total_return'],
             barmode='overlay', opacity=0.7,
             color='Payment Method',
             color_discrete_sequence=px.colors.qualitative.Prism,
             pattern_shape = 'variable',
             category_orders={'Payment Method': payment_method_order},
             title='1. Payment Method Distribution vs. Returns',
             labels={'value': 'Return & Total Order Count'})


# Add overall percentage and returns rate as text on the graph
for i, row in payment_count_returns.iterrows():
    fig1.add_annotation(x=row['Payment Method'], y=row['total_count']-5000, text=f"Total:{row['total_count']/ 1000:,.2f}K<br>({row['percentage']:.2f}%)",
                       font=dict(color='black', size=12), showarrow=False)

for i, row in payment_count_returns.iterrows():
    fig1.add_annotation(x=row['Payment Method'], y=row['total_return']-5000, text=f"Returned:{row['total_return']/ 1000:,.2f}K<br>({row['return_rates']:.2f}%)",
                       font=dict(color='white', size=12), showarrow=False)
    
fig1.update_traces(hovertemplate="Payment Method: %{x}<br>"
                                 "Count: %{y:,.0f}<extra></extra>")

fig1.update_layout(title={'x': 0.5, 'y': 0.9},
                  xaxis_title= None,
                  width=600, height=550,
                  showlegend=False)


## 2. Payment Method by gender and age group
payment_gender_age = df_2.groupby(['Payment Method','Gender','Age Group']).agg(Count=('Customer ID','size')).reset_index()

fig2 = px.bar(payment_gender_age, x='Age Group', y='Count', barmode='group',
              facet_col='Payment Method', #facet_row='Gender',
              color='Gender',
              color_discrete_map={'Female': '#FF6692', 'Male': '#3366CC'},
              category_orders={'Payment Method': payment_method_order},
              title='2. Payment Method Distribution by Gender and Age',
              labels={'Count': 'Number of Customers'},
              text_auto=True)

fig2.update_traces(width=0.5)

fig2.update_layout(title={'x': 0.5, 'y': 0.95},
                   bargap=0.2,
                   xaxis_title= None,
                   xaxis2_title= None,
                   xaxis3_title= None,
                   width=900, height=400,
                   legend_title=None,
                   legend=dict(orientation="h",yanchor="bottom",
                              y=1.14, xanchor="center",x=0.5))


## 3. Payment Method by Segment
payment_segment = df_2.groupby(['Payment Method','Segment Tier']).agg(Count=('Customer ID','size')).reset_index()
payment_segment['Percentage'] = payment_segment['Count']/payment_segment['Count'].sum()

fig3 = px.bar(payment_segment, x='Payment Method', y='Percentage', barmode='group',
              color='Segment Tier', #facet_col='Payment Method',
              color_discrete_sequence=px.colors.qualitative.Dark24,
              category_orders={'Payment Method': payment_method_order, 'Segment Tier': segment_order},
              title='3. Payment Method Distribution by Segment',
              labels={'Percentage': 'Distribution (%)'},
              text_auto='.1%')

fig3.update_layout(title={'x': 0.5, 'y': 0.9},
                   bargap=0.1,
                   xaxis_title= None,
                   width=600, height=450)


###
fig1.show()
fig2.show()
fig3.show()

> **Insights from payment analysis:**
> * Credit Card emerges as the most preferred payment method, while PayPal exhibits the highest return rate among the three.
> * Customer payment behavior patterns remain largely consistent across all age ranges and genders, with some variations observed: PayPal shows slightly higher demand among the 20-29 age group, and credit card and cash usage are more popular among females in their 30s.
> * Payment method distributions appears consistent across all segments, with minor variations where the low segment shows slightly higher usage of Cash and the high segment exhibits a slightly higher preference for PayPal. 

### **4.7 Find correlations**

In [ ]:
## 1. Explore correlations between each quantitative variable
# round p-values to 3 decimals
df2_corr = round(df_2[['Product Price','Quantity','Total Price',
                       'Customer Age','Returns', 'Churn']].corr(method='pearson'), 3)
# Create a correlation heatmap
fig = px.imshow(df2_corr, 
                color_continuous_scale = 'blugrn',
                text_auto=True, aspect="auto")

fig.update_layout(title={'x': 0.5, 'y': 0.9},
                  width=800, height=450)    

fig.show()

> **Insights discovered from the correlation metrix:** 
<br>
> * **Product Price & Total Price**: There is a strong positive correlation (0.72) between Product Price and Total Price. This suggests that as the price of individual products increases, the total price of purchases also tends to increase, which is an expected relationship.
<br>
> *  **Quantity & Total Price**: There's also a moderately strong positive correlation (0.608) between the Quantity of items purchased and the Total Price. This indicates that as more items are purchased, the total price increases.
> *  **Customer Age, Returns, and Churn**: There's no strong linear relationships with other factors, meaning they do not significantly affect on another in this case.  

In [ ]:
## 2. Find correlation between churn and return rate
# Create scatter plot
fig = px.scatter(rates, x='Return_Rate', y='Churn_Rate', 
                 trendline="ols", trendline_scope="overall")

# Calculate correlation coefficient
rates_corr = rates['Return_Rate'].corr(rates['Churn_Rate'])

# Add annotation for correlation coefficient
fig.add_annotation(x=41.5, y=20.15, 
                   text=f'R = {rates_corr:.3f}', 
                   showarrow=False, 
                   font=dict(size=13, color="red"))

fig.show()

This scatter plot shows a weak negative linear relationship between churn rate and return rate, suggesting that as one variable increases, the other tends to decrease slightly. However, the correlation coefficient is close to zero, indicating a very weak relationship and little evidence of correlation between the two variables.

### **5. Conclusion & Recommendation (SHARE)**

#### **⦿ Key Insights**

**Customer Demographics and Behaviors:**
* Spending behaviors are similar across genders, with males slightly outnumbering females.
* The dataset comprises a diverse customer base, primarily aged between 20 and 69.
* Customers aged 20-29 prefer Home goods, while those in the 40-49 age group favor Clothing.
* Payment method preferences are evenly distributed, with credit cards slightly prevailing.

**Product and Sales Trends:**
* Clothing is the most popular category, while Home generates the highest profits. Electronics has the highest return rate, whereas Books generate the lowest sales revenue.
* Sales revenue consistently hovers around $51M per year with minor fluctuations.
* February consistently records the lowest total sales, possibly due to fewer days, but not necessarily the lowest average monthly sales.
* The return rate has remained consistently high at around 40% since 2020, potentially indicating issues with product satisfaction or quality.

**Customer Segmentation based on CLV:**
* The medium segment holds the majority of customers and generates the highest revenue, followed closely by the high segment.
* Churn and return rates across all segments hover around 20% and 40%, respectively, with the medium segment exhibiting the highest rates.
* Age distributions are similar across segments, with the highest concentration observed among customers aged 20-29.
* The low segment shows a slightly higher usage of Cash, while the high segment exhibits a slightly higher preference for PayPal.

#### **⦿ Business & Marketing Recommendations**

**Targeted Marketing Strategies:**
* Tailor marketing campaigns to different customer segments based on their purchasing behavior, CLV, and demographic characteristics like gender and age.
  * Example: Create specific promotions for Home goods aimed at the 20-29 age group and Clothing for the 40-49 age group.
* Offer personalized promotions, discounts, and messaging based on segment characteristics and purchasing behaviors to improve customer engagement and loyalty.
* Introduce seasonal sales initiatives to offset slower or shorter months such as February and September.
  * Example: Offer special discounts or curated product sets for Valentine's Day, catering to the holiday gift market.

**Product & Quality Optimization:**
* Utilize sales data and trends to optimize inventory management practices, ensuring adequate stock levels of high-demand/profit products such as Home and Clothing, while minimizing excess inventory for products with lower sales volumes. 
* Identify and address the root causes of high return rates, particularly for Electronics, to enhance product quality and customer satisfaction.
* Implement bundling strategies to encourage larger purchases and increase revenue by offering complementary products at discounted rates.
  * Example: Provide discounted bundles for purchasing multiple books from the same series. 

**Customer Retention and Loyalty Programs:**
* Implement targeted customer retention initiatives such as loyalty programs or exclusive perks, to reduce churn rates and encourage repeat purchases. 
  * Example: Offer VVIP or member-exclusive benefits and rewards.
* Focus on improving overall customer satisfaction by addressing issues related to product quality, customer service, and delivery experience, which could contribute to the high return rates.

**Payment Method Optimization:**
* Offer incentives or discounts for using credit cards, considering their slight prevalence over other payment methods. Collaborate with banks to offer credit card exclusive reward points or cashback offers.
* Continue to enhance the payment experience by ensuring seamless integration of payment gateways and expand more digital payment options to cater to diverse customer preferences. 
  * Ex: offering intallment plans or accepting crypto-payments. 

**Performance Monitoring and Analysis:**
* Establish regular performance monitoring measurements to track key metrics(KPIs) such as sales revenue, churn rates, return rates, and customer satisfactions.
* Conduct regular quality checks and customer feedback surveys to address any issues promptly and continually improve overall customer experience.
* Perform ongoing analysis and market research to identify emerging trends, patterns, and opportunities for product improvement or innovation.